<a href="https://colab.research.google.com/github/anilaksu/Algorithmic-Trading-Codes/blob/main/Udemy_Triangular_Arbitrage_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Triangular Arbitrage for Crypto with Python Udemy Training**


Anil Aksu

Personal e-mail: aaa293@cornell.edu



**Outline:**


1.  CEFI Triangular Arbitrage
  * Synchronous vs Asynchronous
  * Blocking & Timeouts
  * Scraping with Selenium
  * Asynchronous Functions
  * Asynchronous Iterators
2. DEFI Triangular Arbitrage
  * Coroutine Objects and Async Functions
  * Ways of running coroutines
  * Cancelling coroutines
  * Awaitable Objects



Clone GitHub Repo:

git clone https://github.com/CryptoWizardsNet/poloniex-tri-arb



If you have never used GitHub before or do not have GitHub working on your command line:

https://www.youtube.com/watch?v=CKcqniGu3tA

https://git-scm.com/book/en/v2/Getting-Started-Installing-Git



Google Colab Notebook:

https://colab.research.google.com/drive/1gvX613_dwF5p6Hea9xMYeC-KokE9e9C_?usp=sharing



Poloniex latest docs:

https://docs.poloniex.com/#introduction



Binance latest docs:

https://binance-docs.github.io/apidocs/spot/en/#kline-candlestick-data

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/ColabNotebooks/FinanceAlgorithms
!ls # special shell command to view the files in the home directory of the notebook environment

Mounted at /content/gdrive
/content/gdrive/MyDrive/ColabNotebooks/FinanceAlgorithms
 2013-03-08options.csv	      EURUSD_Options_Data.csv	   OptionsTrading.ipynb
 2013-03-08stocks.csv	      EURUSD_Options_Data.gsheet   PriceJump.gdraw
'Asynchronous Python.ipynb'  'ForEx&IndexData.xls'	  'Stock Markets Codes.ipynb'
 async_scrape.py	      local.csv


#**1. How to pull any API Data**

In [1]:
# Here we install required libraries for API data manipulation
!python3 -V
!which pip3
!pip3 install requests --upgrade --no-cache-dir

Python 3.10.12
/usr/local/bin/pip3


In [2]:
# Here we import required libraries
import requests
import json

In [8]:
# Poloniex prices
prices = requests.get("https://api.poloniex.com/markets/price")
prices_list = []

if prices.status_code == 200:
  prices_json = json.loads(prices.text)
  for p in prices_json:
    prices_list.append({"ticker": p["symbol"], "price" : p["price"]})

In [9]:
print(prices_list)

[{'ticker': 'BTS_BTC', 'price': '0.0000000551'}, {'ticker': 'DASH_BTC', 'price': '0.000567'}, {'ticker': 'DOGE_BTC', 'price': '0.000001704'}, {'ticker': 'LTC_BTC', 'price': '0.001332'}, {'ticker': 'XLM_BTC', 'price': '0.00000213'}, {'ticker': 'XEM_BTC', 'price': '0.00000045'}, {'ticker': 'XMR_BTC', 'price': '0.002264'}, {'ticker': 'XRP_BTC', 'price': '0.00000992'}, {'ticker': 'BTC_USDT', 'price': '56419.68'}, {'ticker': 'DASH_USDT', 'price': '32.82'}, {'ticker': 'LTC_USDT', 'price': '75.142'}, {'ticker': 'XLM_USDT', 'price': '0.1201'}, {'ticker': 'XMR_USDT', 'price': '128.86'}, {'ticker': 'XRP_USDT', 'price': '0.5594'}, {'ticker': 'ETH_BTC', 'price': '0.05766'}, {'ticker': 'ETH_USDT', 'price': '3255.49'}, {'ticker': 'SC_BTC', 'price': '0.0000001642'}, {'ticker': 'DCR_BTC', 'price': '0.000392'}, {'ticker': 'LSK_BTC', 'price': '0.00001513'}, {'ticker': 'STEEM_BTC', 'price': '0.00000449'}, {'ticker': 'ETC_BTC', 'price': '0.000502'}, {'ticker': 'ETC_ETH', 'price': '0.00864'}, {'ticker': 'E

In [ ]:
# Poloniex Orderbook
ticker = "DASH_BTC"
limit = 20
# Structure Orderbook
ask_prices = []
ask_sizes = []
counts = 0

ob = requests.get(f"https://api.poloniex.com/markets/{ticker}/orderBook?limit={limit}")

if ob.status_code == 200:
  ob_json = json.loads(ob.text)
  for a in ob_json["asks"]:
    if counts% 2 == 0:
      ask_prices.append({"ticker": p["symbol"], "price" : p["price"]})
    counts += 1

##1.2 Sync vs Async

It is basically consecutive vs concurrent

**Synchronous Code**

In [ ]:
import asyncio

# This is how we define async function
async def main():
  print('hello...')
  await asyncio.sleep(2) # Here we give a pause for 2 seconds
  print('...world')
  return 'bye'


# To run an asynchronous function, we need to run as follows:
await main()

# Here we create a coroutine object
# result =  main()
# try:
#   result.send(None)
# except StopIteration as e:
#   print('result was:', e.value)

hello...
...world


'bye'

##1.5 Asynchronous Iterators

In [ ]:
# The Asynchronous Generators
import asyncio

async def download(urls):
  for url in urls:
    await asyncio.sleep(1)
    response = {'status': 200, 'data': f'content of {url}'}
    if url == 'bing.com':
      response['status'] = 500
    yield response

async def main():
  urls = [
      'google.com',
      'bing.com',
      'duckduckgo.com'
  ]

  responses = [value async for value in download(urls) if value['status'] != 200]
  print(responses)

await main()

[{'status': 500, 'data': 'content of bing.com'}]


#**2. Coroutines & Awaitables**

##2.1 Coroutines Objects and Async Functions

In [ ]:
import asyncio
import inspect

async def main():
  pass

print(type(main))
print(inspect.iscoroutinefunction(main))
print(dir(main()))

<class 'function'>
True
['__await__', '__class__', '__del__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'close', 'cr_await', 'cr_code', 'cr_frame', 'cr_origin', 'cr_running', 'send', 'throw']


<ipython-input-143-56483b0f906b>:9: RuntimeWarning: coroutine 'main' was never awaited
  print(dir(main()))


##2.2 Ways of Running Coroutines

In [ ]:
import asyncio
import inspect

async def main():
  print('The main function')

# 1. using await
await main()
# 2. creating a task
asyncio.create_task(main())

The main function


<Task pending name='Task-58' coro=<main() running at <ipython-input-145-bc733164109c>:4>>

The main function


##2.3 Cancelling Coroutines

In [ ]:
import asyncio

async def stopwatch():
  count = 0
  while True:
      await asyncio.sleep(1)
      count += 1
      print(count)

async def main():
  task = asyncio.create_task(stopwatch())
  await asyncio.sleep(3)
  task.cancel()   # Here we say "Cancel the task after 3rd second"

await main()

1
2


##2.4 Awaitable Objects

In [ ]:
import asyncio

class Stopwatch:
  def __await__(self):
    yield

async def main():
  await Stopwatch()

await main()

#**3. Tasks, Futures and the Event Loop**

##3.1 Using Task Objects

In [ ]:
import asyncio

async def stopwatch():
  count = 0
  while count < 4:
      await asyncio.sleep(1)
      count += 1
      print(count)

def callb(task):
  print('task is done', task)

async def main():
  task = asyncio.create_task(stopwatch())
  task.set_name('My Task')
  task.add_done_callback(callb)
  print(task.get_name())
  print(task.get_coro())
  print(task.get_name())
  await task

await main()

My Task
<coroutine object stopwatch at 0x7ea2787a4660>
My Task
1
2
3
4
task is done <Task finished name='My Task' coro=<stopwatch() done, defined at <ipython-input-150-b87e2fe8d0a6>:3> result=None>


##3.2 Interacting with the Event Loop:

In [ ]:
import asyncio

async def stopwatch():
  count = 0
  while count < 4:
      await asyncio.sleep(1)
      count += 1
      print(count)

def callb(task):
  print('task is done', task)

async def main():
  print(asyncio.get_running_loop())
  task = asyncio.create_task(stopwatch())
  task.add_done_callback(callb)

  await task

await main()

<_UnixSelectorEventLoop running=True closed=False debug=False>
1
2
3
4
task is done <Task finished name='Task-69' coro=<stopwatch() done, defined at <ipython-input-151-f16058d61056>:3> result=None>


##3.3 The concept of the future:

In [ ]:
import asyncio

async def stopwatch():
  count = 0
  while count < 4:
      await asyncio.sleep(1)
      count += 1
      print(count)

def callb(task):
  print('task is done', task)

async def main():
  task = asyncio.create_task(stopwatch())
  task.add_done_callback(callb)
  print(asyncio.isfuture(task))

  await task

await main()

True
1
2
3
4
task is done <Task finished name='Task-71' coro=<stopwatch() done, defined at <ipython-input-152-fd9b0fb67711>:3> result=None>
